In [ ]:
from src.ssunet.dataloader import BinomDataset, ValidationDataset
from src.ssunet.models import Bit2Bit
from src.ssunet.configs import load_config, example_config_path

# Load the configuration
config = load_config(example_config_path)

# Load the data based on the configuration
data = config.path_config.load_ssunet_data()
ground_truth = config.path_config.load_ground_truth()

model = Bit2Bit(config.model_config)

# Initialize the datasets based on the configuration
train_data = BinomDataset(data, config.data_config, config.split_params)

# Initialize the dataloaders
train_loader = config.loader_config.loader(train_data)

# Initialize the trainer
trainer = config.train_config.trainer

# Print the input size
print(f"input_size: {tuple(next(iter(train_loader))[1].shape)}")

# Train the model
trainer.fit(model, train_loader, train_loader)